## GBDT
> 一种迭代的决策树算法，该算法由多个决策树组成;

**NOTE**: 树模型也分为决策树和回归树，决策树常用来分类问题，回归树常用来预测问题。决策树常用于分类标签值，比如用户性别、网页是否是垃圾页面、用户是不是作弊；而回归树常用于预测真实数值，比如用户的年龄、用户点击的概率、网页相关程度等等。**由于GBDT的核心在与累加所有树的结果作为最终结果，而分类结果对于预测分类并不是这么的容易叠加**（其实并不是简单的叠加，而是每一步每一棵树拟合的残差和选择分裂点评价方式都是经过公式推导得到的），所以GBDT中的树都是**回归树**（其实回归树也能用来做分类）

### 模型思想
GBDT算法可以看做是K棵树组成的加法模型:
$$\hat y_{i} = \sum_{k=1}^K f_{k}(x_{i}), f_{k} \in F$$
其中$F$为所有树组成的函数空间。与一般的机器学习算法不同，加法模型不是学习$d$维空间中的权重，而是直接学习函数(决策树)集合；
上述加法模型的目标函数定义为:
$$Obj=\sum_{i=1}^n l(y_{i}, \hat y_{i}) + \sum_{k=1}^K \Omega(f_{k})$$
其中$\Omega$表示决策树的复杂度，树的复杂度包括**树的节点数量、树的深度或者叶子节点所对应分数的L2范数等**；


## Xgboost
> 一个大规模、分布式的通用Gradient Boosting（GBDT）库，它在Gradient Boosting框架下实现了GBDT和一些广义的线性机器学习算法;

### GBDT 与 Xgboost的区别：
+ 传统GBDT以CART作为基分类器，xgboost还支持线性分类器，这个时候xgboost相当于带L1和L2正则化项的逻辑斯蒂回归（分类问题）或者线性回归（回归问题）；

+ 传统GBDT在优化时只用到一阶导数信息，xgboost则对代价函数进行了二阶泰勒展开，同时用到了一阶和二阶导数。顺便提一下，xgboost工具支持自定义代价函数，只要函数可一阶和二阶求导；

+ xgboost在代价函数里加入了正则项，用于控制模型的复杂度。正则项里包含了树的叶子节点个数、每个叶子节点上输出的score的L2模的平方和。从Bias-variance tradeoff角度来讲，正则项降低了模型的variance，使学习出来的模型更加简单，防止过拟合，这也是xgboost优于传统GBDT的一个特性。

+ Shrinkage（缩减），相当于学习速率（xgboost中的eta）。xgboost在进行完一次迭代后，会将叶子节点的权重乘上该系数，主要是为了削弱每棵树的影响，让后面有更大的学习空间。实际应用中，一般把eta设置得小一点，然后迭代次数设置得大一点。（补充：传统GBDT的实现也有学习速率）

+ 列抽样（column subsampling）。xgboost借鉴了随机森林的做法，支持列抽样，不仅能降低过拟合，还能减少计算，这也是xgboost异于传统gbdt的一个特性。

+ 对缺失值的处理。对于特征的值有缺失的样本，xgboost可以自动学习出它的分裂方向。

+ xgboost工具支持并行。boosting不是一种串行的结构吗?怎么并行的？**注意xgboost的并行不是tree粒度的并行**，xgboost也是一次迭代完才能进行下一次迭代的（第t次迭代的代价函数里包含了前面t-1次迭代的预测值）。**xgboost的并行是在特征粒度上的**。我们知道，决策树的学习最耗时的一个步骤就是对特征的值进行排序（因为要确定最佳分割点），xgboost在训练之前，预先对数据进行了排序，然后保存为block结构，后面的迭代中重复地使用这个结构，大大减小计算量。这个block结构也使得并行成为了可能，在进行节点的分裂时，需要计算每个特征的增益，最终选增益最大的那个特征去做分裂，那么各个特征的增益计算就可以开多线程进行。
